<a href="https://colab.research.google.com/github/yilunz/GIS_Remote_Sensing/blob/master/hw4_Pretrain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
import torchvision
from torchvision import datasets, transforms
from PIL import Image
import torch.distributed as dist

import os
import subprocess
from mpi4py import MPI
import h5py
import time

In [2]:
batch_size=128
LR=0.001
Num_Epochs=100
num_output=100
scheduler_step_size=20
scheduler_gamma=0.3
# c=3
# Kernel_Size=3

transform_train=transforms.Compose([transforms.RandomHorizontalFlip(),transforms.RandomVerticalFlip(),transforms.ToTensor()])
transform_test=transforms.ToTensor()

#Load CIFAR 100 Dataset
trainset=torchvision.datasets.CIFAR100(root='~/scratch/',train=True, download=True,transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset,batch_size=batch_size,shuffle=True,num_workers=8)

testset=torchvision.datasets.CIFAR100(root='~/scratch/',train=False,download=True,transform=transform_test)
testloader=torch.utils.data.DataLoader(testset,batch_size=batch_size,shuffle=False,num_workers=8)



0it [00:00, ?it/s]

169009152it [00:07, 23641801.04it/s]                               


Extracting /root/scratch/cifar-100-python.tar.gz to /root/scratch/
Files already downloaded and verified


In [12]:
def resnet18(pretrained=True):
  model = torchvision.models.resnet.ResNet(torchvision.models.resnet.BasicBlock, [2, 2, 2, 2])
  if pretrained:
    model.load_state_dict(model_zoo.load_url(model_urls['resnet18'],model_dir='./'))
  return model

model=resnet18(pretrained=True)


NameError: ignored

In [0]:
#up sample CIFAR 100 to the scale of 224x224
class pre_resnet(nn.Module):
  def __init__(self):
    super(pre_resnet,self).__init__()
    self.upsample=nn.Upsample(scale_factor=7, mode='bilinear')
    train=resnet18(pretrained=True)
    num_ftrs=train.fc.in_features
    model.fc=nn.Linear(nun_ftrs,output_features)
    self.train=train
  def forward(self,x):
    out=self.upsample(x)
    out=self.train(out)
    return out

In [0]:
# #from colab tutorial
# class PretrainedModel(nn.Module):
#   def __init__(self, output_features):
#     super().__init__()
#     model = models.resnet18(pretrained=True)
#     num_ftrs = model.fc.in_features
#     model.fc = nn.Linear(num_ftrs, output_features)
#     self.model = model
        
#   def forward(self, x):
#     return self.model(x)

In [9]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
preresnet=pre_resnet.to(device)
criterion=nn.CrossEntropyLoss()
optimizer=torch.optim.RMSprop(resnet.parameters(),lr=LR,weight_decay=0.0005)
scheduler=torch.optim.lr_scheduler.StepLR(optimizer,step_size=scheduler_step_size,gamma=scheduler_gamma)

start_time=time.time()
for epoch in range(Num_Epochs):
  preresnet.train()
  scheduler.step()
  total = 0
  correct = 0
  start_time = time.time()
  for images,labels in trainloader:
    images=images.to(device)
    labels=labels.to(device)
    outputs=preresnet(images)
    #print(outputs.shape)
    
    optimizer.zero_grad()
    loss = criterion(outputs, labels)
    _,predicted = torch.max(outputs,1)
    total += labels.size(0)
    
    correct += (predicted == labels).sum().item()
  
    loss.backward()
    optimizer.step()
  
  train_accuracy = correct/total
  
  with torch.no_grad():
    preresnet.eval()
    correct=0
    total=0
    for images,labels in testloader:
      #images,labels = data
      images = images.to(device)
      labels = labels.to(device)
      outputs = preresnet(images)

      total += labels.size(0)
      _,predicted = torch.max(outputs,1)
      correct += (predicted == labels).sum().item()
    test_accuracy = correct/total
  
  print("Epoch {0}, Time {1:.4f}, Train Acc {2:.4f}, Test Acc {3:.4f}".format(epoch, round(time.time()-start_time,4), round(train_accuracy,4), round(test_accuracy,4)))
  torch.save(resnet.state_dict(),'epoch-{}.ckpt'.format(epoch))

AttributeError: ignored